In [ ]:
!pip install pyspark


In [9]:
pip install pandas openpyxl


In [ ]:
# Check if the file exists in the current directory
!ls -l "Online Retail.xlsx"

# List all files in the current directory to see what is available
!ls

ls: cannot access 'Online Retail.xlsx': No such file or directory
sample_data


In [11]:
from google.colab import files
uploaded = files.upload()


Saving Online Retail.xlsx to Online Retail.xlsx


In [12]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("Online Retail.xlsx", engine="openpyxl")

# Save as CSV
df.to_csv("retail_data.csv", index=False)

print("✅ Conversion complete. File saved as retail_data.csv")


✅ Conversion complete. File saved as retail_data.csv


In [13]:
import pandas as pd

# Replace the filename with your uploaded one, if needed
df = pd.read_excel("Online Retail.xlsx", engine="openpyxl")

# Save to CSV
df.to_csv("retail_data.csv", index=False)
print("✅ Excel converted to retail_data.csv")


✅ Excel converted to retail_data.csv


In [14]:
import os
os.listdir()


['.config', 'Online Retail.xlsx', 'retail_data.csv', 'sample_data']

In [15]:
!pip install pyspark


In [17]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RetailSalesAnalysis").getOrCreate()



In [18]:
df = spark.read.csv("retail_data.csv", header=True, inferSchema=True)
df.show(5)
df.printSchema()


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows

roo

In [19]:
from pyspark.sql.functions import col

df_clean = df.filter((col("CustomerID").isNotNull()) & (col("Quantity") > 0))
df_clean.show(5)


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [20]:
from pyspark.sql.functions import sum

df_clean.groupBy("Description") \
    .agg(sum("Quantity").alias("TotalSold")) \
    .orderBy("TotalSold", ascending=False) \
    .show(10, truncate=False)


+----------------------------------+---------+
|Description                       |TotalSold|
+----------------------------------+---------+
|PAPER CRAFT , LITTLE BIRDIE       |80995    |
|MEDIUM CERAMIC TOP STORAGE JAR    |77916    |
|WORLD WAR 2 GLIDERS ASSTD DESIGNS |54415    |
|JUMBO BAG RED RETROSPOT           |46181    |
|WHITE HANGING HEART T-LIGHT HOLDER|36725    |
|ASSORTED COLOUR BIRD ORNAMENT     |35362    |
|PACK OF 72 RETROSPOT CAKE CASES   |33693    |
|POPCORN HOLDER                    |30931    |
|RABBIT NIGHT LIGHT                |27202    |
|MINI PAINT SET VINTAGE            |26076    |
+----------------------------------+---------+
only showing top 10 rows



In [21]:
from pyspark.sql.functions import month

df_clean = df_clean.withColumn("InvoiceMonth", month("InvoiceDate"))

df_clean.groupBy("InvoiceMonth") \
    .agg(sum("Quantity").alias("MonthlySales")) \
    .orderBy("InvoiceMonth") \
    .show()


+------------+------------+
|InvoiceMonth|MonthlySales|
+------------+------------+
|           1|      349147|
|           2|      265638|
|           3|      348544|
|           4|      292225|
|           5|      373685|
|           6|      363699|
|           7|      369432|
|           8|      398938|
|           9|      544899|
|          10|      593908|
|          11|      681888|
|          12|      599693|
+------------+------------+

